In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import tqdm
import joblib

In [2]:
# Динамическое получение текущей рабочей директории
current_path = Path.cwd()  # Получает текущую рабочую директорию
root_directory = current_path.parent

In [3]:

df = pd.read_csv(root_directory / "datasets" / "merged_df.csv")

In [4]:
pd.set_option('display.max_columns', None)

## для Прирост массы кг/день

In [5]:

# Данные о родителях
df_родители = df[["ID_особи", "Пол", "Удой л/день", "Упитанность", 
                   "Коэффициент инбридинга (F)", "Прирост веса кг/день", 
                   "Здоровье (1-10)", "Фертильность (%)", 
                   "Генетическая ценность (баллы)"]]

# Присоединение отцов и матерей
df = df.merge(
    df_родители.add_suffix("_папа"), 
    left_on="Родитель_папа", 
    right_on="ID_особи_папа", 
    how="left"
).merge(
    df_родители.add_suffix("_мама"), 
    left_on="Родитель_мама", 
    right_on="ID_особи_мама", 
    how="left"
)

In [6]:
df.head()

,ID_особи,Пол,Порода,Дата_Рождения,Родитель_папа,Родитель_мама,Удой л/день,Упитанность,Коэффициент инбридинга (F),Прирост веса кг/день,Здоровье (1-10),Фертильность (%),Генетическая ценность (баллы),mutation_id,chrom,pos,ref,alt,Признак,beta,Генотип коровы,mutation_effect,ID_особи_папа,Пол_папа,Удой л/день_папа,Упитанность_папа,Коэффициент инбридинга (F)_папа,Прирост веса кг/день_папа,Здоровье (1-10)_папа,Фертильность (%)_папа,Генетическая ценность (баллы)_папа,ID_особи_мама,Пол_мама,Удой л/день_мама,Упитанность_мама,Коэффициент инбридинга (F)_мама,Прирост веса кг/день_мама,Здоровье (1-10)_мама,Фертильность (%)_мама,Генетическая ценность (баллы)_мама
0,1,Самка,Симентальская,2022-12-22,1996,150,30.464534,4,0.17,1.146774,8,76.0,93,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,9.52,1996.0,Самец,NaN,3.0,0.02,1.09,6.0,76.0,73.0,150.0,Самка,30.151136,4.0,0.02,1.22,5.0,85.0,70.0
1,1,Самка,Симентальская,2022-12-22,1996,150,30.464534,4,0.17,1.146774,8,76.0,93,mut_000001,4,39128359,G,T,Фертильность (%),9.52,T/T,9.52,1996.0,Самец,NaN,3.0,0.02,1.09,6.0,76.0,73.0,150.0,Самка,30.151136,4.0,0.02,1.22,5.0,85.0,70.0
2,1,Самка,Симентальская,2022-12-22,1996,150,30.464534,4,0.17,1.146774,8,76.0,93,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,-6.76,1996.0,Самец,NaN,3.0,0.02,1.09,6.0,76.0,73.0,150.0,Самка,30.151136,4.0,0.02,1.22,5.0,85.0,70.0
3,1,Самка,Симентальская,2022-12-22,1996,150,30.464534,4,0.17,1.146774,8,76.0,93,mut_000002,6,5624123,T,A,Прирост веса кг/день,-6.76,A/A,-6.76,1996.0,Самец,NaN,3.0,0.02,1.09,6.0,76.0,73.0,150.0,Самка,30.151136,4.0,0.02,1.22,5.0,85.0,70.0
4,1,Самка,Симентальская,2022-12-22,1996,150,30.464534,4,0.17,1.146774,8,76.0,93,mut_000003,26,41747388,T,A,Здоровье (1-10),-2.64,T/T,1.00,1996.0,Самец,NaN,3.0,0.02,1.09,6.0,76.0,73.0,150.0,Самка,30.151136,4.0,0.02,1.22,5.0,85.0,70.0


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# Признаки для модели (данные о родителях)
features = [
    "Упитанность_мама", "Коэффициент инбридинга (F)_мама", "Прирост веса кг/день_мама", 
    "Здоровье (1-10)_мама", "Фертильность (%)_мама", "Генетическая ценность (баллы)_мама",
    "Упитанность_папа", "Коэффициент инбридинга (F)_папа", "Прирост веса кг/день_папа", 
    "Здоровье (1-10)_папа", "Фертильность (%)_папа", "Генетическая ценность (баллы)_папа"
]

# Целевая переменная - Прирост веса кг/день
target = "Прирост веса кг/день"

# Подготовка данных для обучения
X = df[features].fillna(0)  # Заполнение пропусков 0
y = df[target]

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тест)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_growth = RandomForestRegressor(random_state=42)
model_growth.fit(X_train, y_train)

# Оценка качества модели
y_pred = model_growth.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


R2: 0.99
MAE: 0.00


In [8]:
def predict_growth(родитель_папа, родитель_мама, model_growth, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Упитанность_мама": мама["Упитанность"],
        "Коэффициент инбридинга (F)_мама": мама["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Генетическая ценность (баллы)_мама": мама["Генетическая ценность (баллы)"],
        "Упитанность_папа": папа["Упитанность"],
        "Коэффициент инбридинга (F)_папа": папа["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Генетическая ценность (баллы)_папа": папа["Генетическая ценность (баллы)"]
    }])
    
    # Предсказание прироста массы
    predicted_growth = model_growth.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Прирост веса кг/день": predicted_growth
    }
    return offspring


In [9]:
# Пример предсказания прироста массы для потомка
offspring_growth = predict_growth(родитель_папа=1996, родитель_мама=150, model_growth=model_growth, df=df)
print(offspring_growth)


{'ID_особи': '150_1996_child', 'Прирост веса кг/день': np.float64(1.1470095838133623)}


In [10]:
# Сохранение модели
joblib.dump(model_growth, root_directory / "models" / "model_growth.pkl")


['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_growth.pkl']

## Фертильность

In [11]:

# Признаки для модели (данные о родителях)
features = [
    "Упитанность_мама", "Коэффициент инбридинга (F)_мама", "Прирост веса кг/день_мама", 
    "Здоровье (1-10)_мама", "Фертильность (%)_мама", "Генетическая ценность (баллы)_мама",
    "Упитанность_папа", "Коэффициент инбридинга (F)_папа", "Прирост веса кг/день_папа", 
    "Здоровье (1-10)_папа", "Фертильность (%)_папа", "Генетическая ценность (баллы)_папа"
]

# Целевая переменная - Фертильность (%)
target = "Фертильность (%)"

# Подготовка данных для обучения
X = df[features].fillna(0)  # Заполнение пропусков 0
y = df[target]

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тест)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_fertility = RandomForestRegressor(random_state=42)
model_fertility.fit(X_train, y_train)

# Оценка качества модели
y_pred = model_fertility.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


R2: 0.99
MAE: 0.07


In [12]:
def predict_fertility(родитель_папа, родитель_мама, model_fertility, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Упитанность_мама": мама["Упитанность"],
        "Коэффициент инбридинга (F)_мама": мама["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Генетическая ценность (баллы)_мама": мама["Генетическая ценность (баллы)"],
        "Упитанность_папа": папа["Упитанность"],
        "Коэффициент инбридинга (F)_папа": папа["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Генетическая ценность (баллы)_папа": папа["Генетическая ценность (баллы)"]
    }])
    
    # Предсказание фертильности
    predicted_fertility = model_fertility.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Фертильность (%)": predicted_fertility
    }
    return offspring


In [13]:
# Пример предсказания фертильности для потомка
offspring_fertility = predict_fertility(родитель_папа=1996, родитель_мама=150, model_fertility=model_fertility, df=df)
offspring_fertility

{'ID_особи': '150_1996_child',
 'Фертильность (%)': np.float64(76.26175780734252)}

In [14]:
# Сохранение модели
joblib.dump(model_fertility, root_directory / "models" / "model_fertility.pkl")

['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_fertility.pkl']

## Здоровье

In [15]:

# Признаки для модели (данные о родителях)
features = [
    "Упитанность_мама", "Коэффициент инбридинга (F)_мама", "Прирост веса кг/день_мама", 
    "Фертильность (%)_мама", "Здоровье (1-10)_мама", "Генетическая ценность (баллы)_мама",
    "Упитанность_папа", "Коэффициент инбридинга (F)_папа", "Прирост веса кг/день_папа", 
    "Фертильность (%)_папа", "Здоровье (1-10)_папа", "Генетическая ценность (баллы)_папа"
]

# Целевая переменная - Здоровье (1-10)
target = "Здоровье (1-10)"

# Подготовка данных для обучения
X = df[features].fillna(0)  # Заполнение пропусков 0
y = df[target]

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тест)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_health = RandomForestRegressor(random_state=42)
model_health.fit(X_train, y_train)

# Оценка качества модели
y_pred = model_health.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


R2: 0.99
MAE: 0.03


In [16]:
def predict_health(родитель_папа, родитель_мама, model_health, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Упитанность_мама": мама["Упитанность"],
        "Коэффициент инбридинга (F)_мама": мама["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Генетическая ценность (баллы)_мама": мама["Генетическая ценность (баллы)"],
        "Упитанность_папа": папа["Упитанность"],
        "Коэффициент инбридинга (F)_папа": папа["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Генетическая ценность (баллы)_папа": папа["Генетическая ценность (баллы)"]
    }])
    
    # Предсказание здоровья
    predicted_health = model_health.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Здоровье (1-10)": predicted_health
    }
    return offspring


In [17]:
# Пример предсказания здоровья для потомка
offspring_health = predict_health(родитель_папа=1996, родитель_мама=150, model_health=model_health, df=df)
print(offspring_health)


{'ID_особи': '150_1996_child', 'Здоровье (1-10)': np.float64(7.97)}


In [18]:
# Сохранение модели
joblib.dump(model_health, root_directory / "models" / "model_health.pkl")

['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_health.pkl']

## Коэффициент инбридинга

In [19]:

# Признаки для модели (данные о родителях)
features = [
    "Упитанность_мама", "Прирост веса кг/день_мама", "Фертильность (%)_мама", 
    "Здоровье (1-10)_мама", "Генетическая ценность (баллы)_мама",
    "Упитанность_папа", "Прирост веса кг/день_папа", "Фертильность (%)_папа", 
    "Здоровье (1-10)_папа", "Генетическая ценность (баллы)_папа"
]

# Целевая переменная - Коэффициент инбридинга (F)
target = "Коэффициент инбридинга (F)"

# Подготовка данных для обучения
X = df[features].fillna(0)  # Заполнение пропусков 0
y = df[target]

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тест)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_inbreeding = RandomForestRegressor(random_state=42)
model_inbreeding.fit(X_train, y_train)

# Оценка качества модели
y_pred = model_inbreeding.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


R2: 0.99
MAE: 0.00


In [20]:
def predict_inbreeding(родитель_папа, родитель_мама, model_inbreeding, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Упитанность_мама": мама["Упитанность"],
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Генетическая ценность (баллы)_мама": мама["Генетическая ценность (баллы)"],
        "Упитанность_папа": папа["Упитанность"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Генетическая ценность (баллы)_папа": папа["Генетическая ценность (баллы)"]
    }])
    
    # Предсказание коэффициента инбридинга (F)
    predicted_inbreeding = model_inbreeding.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Коэффициент инбридинга (F)": predicted_inbreeding
    }
    return offspring


In [21]:
# Пример предсказания коэффициента инбридинга для потомка
offspring_inbreeding = predict_inbreeding(родитель_папа=1996, родитель_мама=150, model_inbreeding=model_inbreeding, df=df)
print(offspring_inbreeding)


{'ID_особи': '150_1996_child', 'Коэффициент инбридинга (F)': np.float64(0.1656)}


In [22]:
# Сохранение модели
joblib.dump(model_inbreeding, root_directory / "models" / "model_inbreeding.pkl")

['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_inbreeding.pkl']

## Упитанность

In [23]:

# Признаки для модели (данные о родителях)
features = [
    "Прирост веса кг/день_мама", "Фертильность (%)_мама", 
    "Здоровье (1-10)_мама", "Генетическая ценность (баллы)_мама",
    "Прирост веса кг/день_папа", "Фертильность (%)_папа", 
    "Здоровье (1-10)_папа", "Генетическая ценность (баллы)_папа"
]

# Целевая переменная - Упитанность
target = "Упитанность"

# Подготовка данных для обучения
X = df[features].fillna(0)  # Заполнение пропусков 0
y = df[target]

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тест)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_upitannost = RandomForestRegressor(random_state=42)
model_upitannost.fit(X_train, y_train)

# Оценка качества модели
y_pred = model_upitannost.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


R2: 0.99
MAE: 0.01


In [24]:
def predict_upitannost(родитель_папа, родитель_мама, model_upitannost, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Генетическая ценность (баллы)_мама": мама["Генетическая ценность (баллы)"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Генетическая ценность (баллы)_папа": папа["Генетическая ценность (баллы)"]
    }])
    
    # Предсказание упитанности
    predicted_upitannost = model_upitannost.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Упитанность": predicted_upitannost
    }
    return offspring


In [25]:
# Пример предсказания упитанности для потомка
offspring_upitannost = predict_upitannost(родитель_папа=1996, родитель_мама=150, model_upitannost=model_upitannost, df=df)
print(offspring_upitannost)


{'ID_особи': '150_1996_child', 'Упитанность': np.float64(3.96)}


In [26]:
# Сохранение модели
joblib.dump(model_upitannost, root_directory / "models" / "model_upitannost.pkl")

['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_upitannost.pkl']

## Генетическая ценность

In [27]:

# Признаки для модели (данные о родителях)
features = [
    "Прирост веса кг/день_мама", "Фертильность (%)_мама", 
    "Здоровье (1-10)_мама", "Упитанность_мама",
    "Прирост веса кг/день_папа", "Фертильность (%)_папа", 
    "Здоровье (1-10)_папа", "Упитанность_папа"
]

# Целевая переменная - Генетическая ценность (баллы)
target = "Генетическая ценность (баллы)"

# Подготовка данных для обучения
X = df[features].fillna(0)  # Заполнение пропусков 0
y = df[target]

# Разделение данных на обучающую и тестовую выборки (80% на обучение, 20% на тест)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели
model_genetic_value = RandomForestRegressor(random_state=42)
model_genetic_value.fit(X_train, y_train)

# Оценка качества модели
y_pred = model_genetic_value.predict(X_test)
print(f"R2: {r2_score(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")


R2: 0.98
MAE: 0.19


In [28]:
def predict_genetic_value(родитель_папа, родитель_мама, model_genetic_value, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Упитанность_мама": мама["Упитанность"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Упитанность_папа": папа["Упитанность"]
    }])
    
    # Предсказание генетической ценности
    predicted_genetic_value = model_genetic_value.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Генетическая ценность (баллы)": predicted_genetic_value
    }
    return offspring


In [29]:
# Пример предсказания генетической ценности для потомка
offspring_genetic_value = predict_genetic_value(родитель_папа=1996, родитель_мама=150, model_genetic_value=model_genetic_value, df=df)
print(offspring_genetic_value)


{'ID_особи': '150_1996_child', 'Генетическая ценность (баллы)': np.float64(92.86)}


In [30]:
# Сохранение модели
joblib.dump(model_genetic_value, root_directory / "models" / "model_genetic_value.pkl")

['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_genetic_value.pkl']

## Удой

In [31]:
# Отфильтровываем только самок
df_females = df[df["Пол"] == "Самка"]

# Признаки для модели
features_udoy = [
    "Упитанность_мама", "Коэффициент инбридинга (F)_мама", "Прирост веса кг/день_мама", 
    "Здоровье (1-10)_мама", "Фертильность (%)_мама", "Генетическая ценность (баллы)_мама",
    "Упитанность_папа", "Коэффициент инбридинга (F)_папа", "Прирост веса кг/день_папа", 
    "Здоровье (1-10)_папа", "Фертильность (%)_папа", "Генетическая ценность (баллы)_папа"
]

# Целевая переменная - Удой л/день
target_udoy = "Удой л/день"

# Исключаем строки с пропущенными значениями в целевой переменной
df_females = df_females.dropna(subset=[target_udoy])

# Подготовка данных для обучения
X_udoy = df_females[features_udoy].fillna(0)  # Заполнение пропусков 0
y_udoy = df_females[target_udoy]

# Разделение данных на обучающую и тестовую выборки
X_train_udoy, X_test_udoy, y_train_udoy, y_test_udoy = train_test_split(X_udoy, y_udoy, test_size=0.2, random_state=42)

# Обучение модели
model_udoy = RandomForestRegressor(random_state=42)
model_udoy.fit(X_train_udoy, y_train_udoy)

# Оценка модели
y_pred_udoy = model_udoy.predict(X_test_udoy)
print(f"R2: {r2_score(y_test_udoy, y_pred_udoy):.2f}")
print(f"MAE: {mean_absolute_error(y_test_udoy, y_pred_udoy):.2f}")


R2: 0.98
MAE: 0.08


In [32]:
def predict_udoy(родитель_папа, родитель_мама, model_udoy, df):
    # Данные о родителях
    parent_data = df[(df["ID_особи"] == родитель_папа) | (df["ID_особи"] == родитель_мама)]
    
    if parent_data.shape[0] < 2:
        raise ValueError("Не найдены оба родителя в таблице!")
    
    мама = parent_data[parent_data["ID_особи"] == родитель_мама].iloc[0]
    папа = parent_data[parent_data["ID_особи"] == родитель_папа].iloc[0]
    
    # Формируем признаки для предсказания
    input_data = pd.DataFrame([{
        "Упитанность_мама": мама["Упитанность"],
        "Коэффициент инбридинга (F)_мама": мама["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_мама": мама["Прирост веса кг/день"],
        "Здоровье (1-10)_мама": мама["Здоровье (1-10)"],
        "Фертильность (%)_мама": мама["Фертильность (%)"],
        "Генетическая ценность (баллы)_мама": мама["Генетическая ценность (баллы)"],
        "Упитанность_папа": папа["Упитанность"],
        "Коэффициент инбридинга (F)_папа": папа["Коэффициент инбридинга (F)"],
        "Прирост веса кг/день_папа": папа["Прирост веса кг/день"],
        "Здоровье (1-10)_папа": папа["Здоровье (1-10)"],
        "Фертильность (%)_папа": папа["Фертильность (%)"],
        "Генетическая ценность (баллы)_папа": папа["Генетическая ценность (баллы)"]
    }])
    
    # Предсказание удоя
    predicted_udoy = model_udoy.predict(input_data)[0]
    
    # Формирование данных о потомке
    offspring = {
        "ID_особи": f"{родитель_мама}_{родитель_папа}_child",  # ID потомка
        "Пол": "Самка",  # Пол фиксируем как женский
        "Удой л/день": predicted_udoy
    }
    return offspring


In [33]:
# Пример предсказания удоя
offspring_udoy = predict_udoy(родитель_папа=1996, родитель_мама=150, model_udoy=model_udoy, df=df)
print(offspring_udoy)


{'ID_особи': '150_1996_child', 'Пол': 'Самка', 'Удой л/день': np.float64(30.464534091485472)}


In [34]:
# Сохранение модели
joblib.dump(model_udoy, root_directory / "models" / "model_udoy.pkl")

['/Users/holodkovaelizavetaigorevna/Downloads/hakaton/models/model_udoy.pkl']